In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import seaborn as sns
import folium


In [2]:
# Load and preprocess data
def preprocess_data():
    # Load datasets
    merged_data = pd.read_csv('final_dataset.csv')
    crop_production = pd.read_csv('crop_production.csv')
    district_population = pd.read_csv('district_level_population.csv')
    food = pd.read_csv('food.csv')
    nutrition = pd.read_csv('nutrition.csv')
    rainfall = pd.read_csv('rainfall.csv')
    yield_complete = pd.read_csv('yield_complete.csv')
    necessary_nutrition = pd.read_csv('necessary_nutrition.csv')
    prices_data = pd.read_csv('prices_districtwise.csv')
    prices_data.columns = prices_data.columns.str.lower()
    prices_data = prices_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Convert date column to datetime
    prices_data['arrival_date'] = pd.to_datetime(prices_data['arrival_date'], format='%d/%m/%Y')
    

    merged_data = pd.merge(merged_data, prices_data, left_on=['district_name'], right_on=['district'])
    merged_data.fillna(method='ffill', inplace=True)

    # Create new features: average_price
    merged_data['average_price'] = merged_data[['min_price', 'max_price']].mean(axis=1)
    
    return merged_data, necessary_nutrition, food

# Preprocess the data
merged_data, necessary_nutrition, food = preprocess_data()


/tmp/ipykernel_4925/2725473808.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prices_data = prices_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_4925/2725473808.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)


In [3]:
merged_data

,state_name,district_name,crop_year,season,crop,area,production,dist code,year_x,state code,...,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price,average_price
0,andhra pradesh,kurnool,1997,kharif,arhar/tur,20900.0,6400.0,51,2019,1,...,andhra pradesh,kurnool,banaganapalli,bengal gram(gram)(whole),desi (whole),2019-03-04,4500,4700,4600,4600.0
1,andhra pradesh,kurnool,1997,kharif,arhar/tur,20900.0,6400.0,51,2019,1,...,andhra pradesh,kurnool,banaganapalli,jowar(sorghum),jowar ( white),2019-03-04,1800,1950,1900,1875.0
2,andhra pradesh,kurnool,1997,kharif,arhar/tur,20900.0,6400.0,51,2019,1,...,andhra pradesh,kurnool,banaganapalli,paddy(dhan)(common),sona mahsuri,2019-03-04,1900,2000,1950,1950.0
3,andhra pradesh,kurnool,1997,kharif,bajra,11600.0,9300.0,51,2019,1,...,andhra pradesh,kurnool,banaganapalli,bengal gram(gram)(whole),desi (whole),2019-03-04,4500,4700,4600,4600.0
4,andhra pradesh,kurnool,1997,kharif,bajra,11600.0,9300.0,51,2019,1,...,andhra pradesh,kurnool,banaganapalli,jowar(sorghum),jowar ( white),2019-03-04,1800,1950,1900,1875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816687,west bengal,nadia,2014,winter,rice,94963.0,273137.0,803,2019,13,...,west bengal,nadia,nadia,onion,other,2019-03-04,1100,1300,1200,1200.0
816688,west bengal,nadia,2014,winter,rice,94963.0,273137.0,803,2019,13,...,west bengal,nadia,nadia,paddy(dhan)(common),fine,2019-03-04,1750,1780,1770,1765.0
816689,west bengal,nadia,2014,winter,rice,94963.0,273137.0,803,2019,13,...,west bengal,nadia,nadia,potato,other,2019-03-04,570,590,580,580.0
816690,west bengal,nadia,2014,winter,rice,94963.0,273137.0,803,2019,13,...,west bengal,nadia,nadia,rice,fine,2019-03-04,3700,3900,3800,3800.0


In [4]:
columns = merged_data.columns.tolist()
for col in columns:
    print(col)


state_name
district_name
crop_year
season
crop
area
production
dist code
year_x
state code
state name
dist name
adult male crossbred (1000 number)
adult male indigenous (1000 number)
adult female crossbred (1000 number)
adult female indigenous (1000 number)
young male crossbred (1000 number)
young male indigenous (1000 number)
young female crossbred (1000 number)
young female indigenous (1000 number)
st_code
state/region
actual rainfall
normal rainfall
dist.code
year_y
state.code
state.name
dist.name
rice.area..1000.ha.
rice.production..1000.tons.
rice.yield..kg.per.ha.
wheat.area..1000.ha.
wheat.production..1000.tons.
wheat.yield..kg.per.ha.
kharif.sorghum.area..1000.ha.
kharif.sorghum.production..1000.tons.
kharif.sorghum.yield..kg.per.ha.
rabi.sorghum.area..1000.ha.
rabi.sorghum.production..1000.tons.
rabi.sorghum.yield..kg.per.ha.
sorghum.area..1000.ha.
sorghum.production..1000.tons.
sorghum.yield..kg.per.ha.
pearl.millet.area..1000.ha.
pearl.millet.production..1000.tons.
pearl.mil

In [5]:
# Define age and gender groups and nutritional requirements
age_gender_groups = {
    'children_male': ['young male crossbred (1000 number)', 'young male indigenous (1000 number)'],
    'children_female': ['young female crossbred (1000 number)', 'young female indigenous (1000 number)'],
    'adults_male': ['adult male crossbred (1000 number)', 'adult male indigenous (1000 number)'],
    'adults_female': ['adult female crossbred (1000 number)', 'adult female indigenous (1000 number)']
}

required_nutrition = necessary_nutrition.set_index('nutrient')['daily_requirement'].to_dict()

def calculate_dietary_needs(state, district):
    input_data = merged_data[(merged_data['state_name'].str.lower() == state.lower()) & (merged_data['district_name'].str.lower() == district.lower())]
    if input_data.empty:
        return "No data available for the given state and district."

    # Predict dietary needs based on population data
    dietary_needs = {}
    
    for nutrient, daily_requirement in required_nutrition.items():
        dietary_needs[nutrient] = {}
        for group, columns in age_gender_groups.items():
            group_population = sum(input_data[col].values[0] for col in columns) * 1000  # Convert thousands to actual number
            total_requirement = group_population * daily_requirement
            dietary_needs[nutrient][group] = total_requirement
    
    return dietary_needs

# Calculate dietary needs
dietary_needs = calculate_dietary_needs('andhra pradesh', 'kurnool')
print(dietary_needs)


{'protein': {'children_male': 1277499.9999999998, 'children_female': 3807500.0, 'adults_male': 8240500.0, 'adults_female': 5026999.999999999}, 'carbohydrates': {'children_male': 7664999.999999999, 'children_female': 22845000.0, 'adults_male': 49443000.0, 'adults_female': 30161999.999999996}, 'total fat': {'children_male': 1788499.9999999998, 'children_female': 5330500.0, 'adults_male': 11536700.0, 'adults_female': 7037799.999999999}, 'saturated fat': {'children_male': 510999.99999999994, 'children_female': 1523000.0, 'adults_male': 3296200.0, 'adults_female': 2010799.9999999998}, 'fiber': {'children_male': 638749.9999999999, 'children_female': 1903750.0, 'adults_male': 4120250.0, 'adults_female': 2513499.9999999995}, 'sugar': {'children_male': 2299499.9999999995, 'children_female': 6853500.0, 'adults_male': 14832900.0, 'adults_female': 9048599.999999998}, 'vitamin a': {'children_male': 22994999.999999996, 'children_female': 68535000.0, 'adults_male': 148329000.0, 'adults_female': 90485

In [6]:
def get_nutrition_info():
    nutrient_to_category = {
        'protein': 'protein',
        'carbohydrates': 'carbohydrate',
        'total fat': 'fat',
        'saturated fat': 'saturated fat',
        'fiber': 'fiber',
        'sugar': 'sugar',
        'vitamin a': 'vitamin a',
        'vitamin c': 'vitamin c',
        'calcium': 'calcium',
        'iron': 'iron',
        'potassium': 'potassium',
        'magnesium': 'magnesium',
        'zinc': 'zinc',
        'vitamin d': 'vitamin d',
        'vitamin b12': 'vitamin b12'
    }
    
    nutrition_info = {}
    for nutrient, category in nutrient_to_category.items():
        food_items = food[food['category'].str.contains(category, case=False, na=False)]
        description = necessary_nutrition[necessary_nutrition['nutrient'].str.lower() == nutrient.lower()]['description'].values[0]
        if not food_items.empty:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': food_items['description'].tolist()
            }
        else:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': ["No specific food items found"]
            }
    return nutrition_info

# Get nutrition information
nutrition_info = get_nutrition_info()
print(nutrition_info)


{'protein': {'daily_requirement': 50.0, 'description': 'essential for growth and repair of tissues', 'food_items': ['textured vegetable protein, dry', 'nutrition bar (snickers marathon protein bar)', 'nutrition bar (south beach living high protein bar)', 'cereal (general mills cheerios protein)', 'nutritional powder mix (eas whey protein powder)', 'nutritional powder mix (eas soy protein powder)', "nutritional powder mix (kellogg's special k20 protein water)"]}, 'carbohydrates': {'daily_requirement': 300.0, 'description': 'primary source of energy', 'food_items': ['No specific food items found']}, 'total fat': {'daily_requirement': 70.0, 'description': 'required for energy and cell function', 'food_items': ['fat back, cooked', "cereal (kellogg's low fat granola)", 'banana juice with lowfat yogurt, baby food', 'mixed fruit juice with lowfat yogurt, baby food', 'table fat, nfs', 'oil or table fat, nfs', 'animal fat or drippings']}, 'saturated fat': {'daily_requirement': 20.0, 'descriptio

In [7]:
def get_nutrition_info(required_nutrition):
    nutrient_to_category = {
        'protein': 'protein',
        'carbohydrates': 'carbohydrate',
        'total fat': 'fat',
        'saturated fat': 'saturated fat',
        'fiber': 'fiber',
        'sugar': 'sugar',
        'vitamin a': 'vitamin a',
        'vitamin c': 'vitamin c',
        'calcium': 'calcium',
        'iron': 'iron',
        'potassium': 'potassium',
        'magnesium': 'magnesium',
        'zinc': 'zinc',
        'vitamin d': 'vitamin d',
        'vitamin b12': 'vitamin b12'
    }
    
    nutrition_info = {}
    for nutrient, category in nutrient_to_category.items():
        food_items = food[food['category'].str.contains(category, case=False, na=False)]
        description = necessary_nutrition[necessary_nutrition['nutrient'].str.lower() == nutrient.lower()]['description'].values[0]
        if not food_items.empty:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': food_items['description'].tolist()
            }
        else:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': ["No specific food items found"]
            }
    return nutrition_info

# Define the required_nutrition dictionary
required_nutrition = necessary_nutrition.set_index('nutrient')['daily_requirement'].to_dict()

# Get nutrition information
nutrition_info = get_nutrition_info(required_nutrition)
print(nutrition_info)


{'protein': {'daily_requirement': 50.0, 'description': 'essential for growth and repair of tissues', 'food_items': ['textured vegetable protein, dry', 'nutrition bar (snickers marathon protein bar)', 'nutrition bar (south beach living high protein bar)', 'cereal (general mills cheerios protein)', 'nutritional powder mix (eas whey protein powder)', 'nutritional powder mix (eas soy protein powder)', "nutritional powder mix (kellogg's special k20 protein water)"]}, 'carbohydrates': {'daily_requirement': 300.0, 'description': 'primary source of energy', 'food_items': ['No specific food items found']}, 'total fat': {'daily_requirement': 70.0, 'description': 'required for energy and cell function', 'food_items': ['fat back, cooked', "cereal (kellogg's low fat granola)", 'banana juice with lowfat yogurt, baby food', 'mixed fruit juice with lowfat yogurt, baby food', 'table fat, nfs', 'oil or table fat, nfs', 'animal fat or drippings']}, 'saturated fat': {'daily_requirement': 20.0, 'descriptio

In [8]:
def identify_surplus_deficit(state):
    state_data = merged_data[merged_data['state_name'].str.lower() == state.lower()]
    if state_data.empty:
        print(f"No data available for the state: {state}")
        return {}

    total_production = state_data.groupby('district_name')['production'].sum()
    if total_production.empty:
        print(f"No production data available for the state: {state}")
        return {}

    dietary_needs = {}
    for district in state_data['district_name'].unique():
        dietary_needs[district] = calculate_dietary_needs(state, district)

    surplus_deficit = {}
    for district, needs in dietary_needs.items():
        if district not in total_production:
            print(f"No production data available for the district: {district}")
            continue

        production = total_production[district] * 1000  # Convert to actual number
        surplus_deficit[district] = {}

        for nutrient, group_needs in needs.items():
            total_need = sum(group_needs.values())
            surplus_deficit[district][nutrient] = production - total_need

    return surplus_deficit

# Identify surplus and deficit areas
surplus_deficit_status = identify_surplus_deficit('andhra pradesh')
print(surplus_deficit_status)


{'kurnool': {'protein': 105504517500.0, 'carbohydrates': 105412755000.0, 'total fat': 105497176500.0, 'saturated fat': 105515529000.0, 'fiber': 105513693750.0, 'sugar': 105489835500.0, 'vitamin a': 105192525000.0, 'vitamin c': 105489835500.0, 'calcium': 105155820000.0, 'iron': 105516263100.0, 'potassium': 103797735000.0, 'magnesium': 105376050000.0, 'zinc': 105518832450.0, 'vitamin d': 105515529000.0, 'vitamin b12': 105521989080.0}, 'west godavari': {'protein': 4288540692500.0, 'carbohydrates': 4288490415000.0, 'total fat': 4288536670300.0, 'saturated fat': 4288546725800.0, 'fiber': 4288545720250.0, 'sugar': 4288532648100.0, 'vitamin a': 4288369749000.0, 'vitamin c': 4288532648100.0, 'calcium': 4288349638000.0, 'iron': 4288547128020.0, 'potassium': 4287605531000.0, 'magnesium': 4288470304000.0, 'zinc': 4288548535790.0, 'vitamin d': 4288546725800.0, 'vitamin b12': 4288550265336.0}}


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

def train_crop_production_model(data, features, target):
    X = data[features]
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Mean Absolute Error: {mae}')
    
    return model

# Features and target
features = ['area', 'actual rainfall', 'normal rainfall', 'crop_year', 'average_price']
target = 'production'

# Train the model
crop_production_model = train_crop_production_model(merged_data, features, target)

# Predict future crop production
future_data = pd.DataFrame({
    'area': [2000],
    'actual rainfall': [310],
    'normal rainfall': [250],
    'crop_year': [2025],
    'average_price': [800]  # Example average price
})
future_prediction = crop_production_model.predict(future_data)
print(f'Predicted Production for 2025: {future_prediction[0]}')


Mean Absolute Error: 212902.72302661158
Predicted Production for 2025: 1320.4242063492059


In [10]:
def define_nutrient_deficiency(data, threshold=5000):
    # Calculate nutrient requirements and production surplus/deficit
    age_gender_groups = {
        'children_male': ['young male crossbred (1000 number)', 'young male indigenous (1000 number)'],
        'children_female': ['young female crossbred (1000 number)', 'young female indigenous (1000 number)'],
        'adults_male': ['adult male crossbred (1000 number)', 'adult male indigenous (1000 number)'],
        'adults_female': ['adult female crossbred (1000 number)', 'adult female indigenous (1000 number)']
    }

    required_nutrition = necessary_nutrition.set_index('nutrient')['daily_requirement'].to_dict()
    data['nutrient_deficiency'] = 0  # Initialize with no deficiency

    for idx, row in data.iterrows():
        total_deficit = 0
        for nutrient, daily_requirement in required_nutrition.items():
            total_requirement = 0
            for group, columns in age_gender_groups.items():
                group_population = sum(row[col] for col in columns) * 1000  # Convert thousands to actual number
                total_requirement += group_population * daily_requirement
            production = row['production'] * 1000  # Convert thousands to actual number
            total_deficit += max(0, total_requirement - production)
        
        # Mark deficiency if total deficit exceeds threshold
        if total_deficit > threshold:
            data.at[idx, 'nutrient_deficiency'] = 1

    return data

# Define nutrient deficiency in the merged dataset
merged_data = define_nutrient_deficiency(merged_data)
print(merged_data[['district_name', 'nutrient_deficiency']].head())


  district_name  nutrient_deficiency
0       kurnool                    1
1       kurnool                    1
2       kurnool                    1
3       kurnool                    1
4       kurnool                    1


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_nutrient_deficiency_model(data, features, target):
    X = data[features]
    y = data[target]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    
    return model, scaler

# Features and target for nutrient deficiency prediction
features = ['area', 'actual rainfall', 'normal rainfall', 'crop_year', 'average_price']
target = 'nutrient_deficiency'

# Train the nutrient deficiency model
nutrient_deficiency_model, scaler = train_nutrient_deficiency_model(merged_data, features, target)

# Example prediction for future data
future_data = pd.DataFrame({
    'area': [2000],
    'actual rainfall': [310],
    'normal rainfall': [250],
    'crop_year': [2025],
    'average_price': [800]  # Example average price
})

# Scale future data
future_data_scaled = scaler.transform(future_data)
deficiency_prediction = nutrient_deficiency_model.predict(future_data_scaled)
print(f'Nutrient Deficiency Prediction for 2025: {deficiency_prediction[0]}')


Accuracy: 0.9663827989641176
Nutrient Deficiency Prediction for 2025: 1


In [15]:
from scipy.optimize import linprog

def optimize_crop_selection(data, demand):
    # Calculate profitability (e.g., production * average_price)
    data['profitability'] = data['production'] * data['average_price']

    # Define optimization problem
    c = -data['profitability'].values  # Negative for maximization
    A_eq = np.ones((1, len(data)))  # Constraint matrix
    b_eq = [demand]  # Total demand to meet
    bounds = [(0, None)] * len(data)  # Bounds for each variable
    
    # Linear programming optimization
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    return result

# Example usage
demand = 10000  # Example total demand
optimization_result = optimize_crop_selection(merged_data, demand)
print(optimization_result)


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -2.0241e+17
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 1
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 2.024e+13  2.024e+13 ...  2.024e+13
                              2.024e+13]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [-2.024e+13]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


In [18]:
def comprehensive_model(state, district):
    # Preprocess Data
    merged_data, necessary_nutrition, food = preprocess_data()

    # Calculate Dietary Needs
    dietary_needs = calculate_dietary_needs(state, district)
    
    # Define the required_nutrition dictionary
    required_nutrition = necessary_nutrition.set_index('nutrient')['daily_requirement'].to_dict()
    
    # Get Nutrition Info
    nutrition_info = get_nutrition_info(required_nutrition)
    
    # Identify Surplus and Deficit Areas
    surplus_deficit_status = identify_surplus_deficit(state)
    
    # Identify States and Districts for Food Exchange
    food_exchange_recommendations = {}
    for nutrient, total_need_dict in dietary_needs.items():
        total_need = sum(total_need_dict.values())
        total_surplus = surplus_deficit_status[district].get(nutrient, 0)
        if total_need > total_surplus:
            # Find state and district with higher production of the needed food
            for other_district in surplus_deficit_status:
                if surplus_deficit_status[other_district].get(nutrient, 0) > total_need - total_surplus:
                    food_exchange_recommendations[nutrient] = other_district
                    break
    
    # Train Crop Production Model with Prices Data
    features = ['area', 'actual rainfall', 'normal rainfall', 'crop_year', 'average_price']
    target = 'production'
    crop_production_model = train_crop_production_model(merged_data, features, target)
    
    # Future Crop Production and Price Forecast
    future_years = [2025, 2026, 2027, 2028, 2029]
    future_production_forecasts = []
    future_price_forecasts = []
    for year in future_years:
        future_data = pd.DataFrame({
            'area': [2000],
            'actual rainfall': [310],
            'normal rainfall': [250],
            'crop_year': [year],
            'average_price': [800]  # Example average price
        })
        future_production = crop_production_model.predict(future_data)
        future_price = 800 + (year - 2025) * 10  # Example linear price increase
        future_production_forecasts.append(future_production[0])
        future_price_forecasts.append(future_price)
    
    # Define and Train Nutrient Deficiency Model
    merged_data = define_nutrient_deficiency(merged_data)
    target = 'nutrient_deficiency'
    nutrient_deficiency_model, scaler = train_nutrient_deficiency_model(merged_data, features, target)
    
    # Prediction example
    future_data_scaled = scaler.transform(future_data)
    deficiency_prediction = nutrient_deficiency_model.predict(future_data_scaled)
    
    # Calculate total demand
    total_demand = 0
    for nutrient, total_need_dict in dietary_needs.items():
        total_need = sum(total_need_dict.values())
        total_surplus = surplus_deficit_status[district].get(nutrient, 0)
        total_demand += total_need - total_surplus

    # Optimize Crop Selection and Distribution for Profitability
    optimization_result = optimize_crop_selection(merged_data, total_demand)
    
    # Combine Results
    results = {
        'Nutrient Deficiencies': dietary_needs,
        'Food Recommendations': nutrition_info,
        'Surplus/Deficit Status': surplus_deficit_status,
        'Food Exchange Recommendations': food_exchange_recommendations,
        'Future Crop Production Forecasts': future_production_forecasts,
        'Future Crop Price Forecasts': future_price_forecasts,
        'Nutrient Deficiency Prediction': deficiency_prediction,
        'Optimization Result': optimization_result
    }
    
    return results

# Example usage
results = comprehensive_model('andhra pradesh', 'kurnool')
print(results)


/tmp/ipykernel_4925/2725473808.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prices_data = prices_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_4925/2725473808.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)


Mean Absolute Error: 214117.7439549588
Accuracy: 0.9663827989641176
{'Nutrient Deficiencies': {'protein': {'children_male': 1277499.9999999998, 'children_female': 3807500.0, 'adults_male': 8240500.0, 'adults_female': 5026999.999999999}, 'carbohydrates': {'children_male': 7664999.999999999, 'children_female': 22845000.0, 'adults_male': 49443000.0, 'adults_female': 30161999.999999996}, 'total fat': {'children_male': 1788499.9999999998, 'children_female': 5330500.0, 'adults_male': 11536700.0, 'adults_female': 7037799.999999999}, 'saturated fat': {'children_male': 510999.99999999994, 'children_female': 1523000.0, 'adults_male': 3296200.0, 'adults_female': 2010799.9999999998}, 'fiber': {'children_male': 638749.9999999999, 'children_female': 1903750.0, 'adults_male': 4120250.0, 'adults_female': 2513499.9999999995}, 'sugar': {'children_male': 2299499.9999999995, 'children_female': 6853500.0, 'adults_male': 14832900.0, 'adults_female': 9048599.999999998}, 'vitamin a': {'children_male': 2299499